### Neural Nets

In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv(r"C:\Users\varun george\OneDrive\Desktop\SNU\4th sem\mlt\datasets\Bank_Personal_Loan_Modelling.csv")

In [5]:
df = df.drop(columns = ["ID","ZIP Code"])
x = df.drop("Personal Loan",axis=1)
y = df["Personal Loan"]

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()

x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.3,random_state=0)

x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

In [6]:
import keras
from keras.models import Sequential
from keras.layers import Dense

from keras.optimizers import Adam

optimizer = Adam(lr=0.01)
loss = 'binary_crossentropy'

C:\Users\varun george\anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [40]:
model = Sequential()
model.add(Dense(units=16, activation='relu', input_dim=x_train.shape[1]))
model.add(Dense(units=1, activation='sigmoid'))

In [41]:
def fitness(weights):
    weights = weights.flatten()
    w0 = weights[:176].reshape((11, 16))
    b0= np.array([0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.])
    model.layers[0].set_weights([w0, b0])
    
    model.compile(optimizer=optimizer, loss=loss)
    Loss = model.evaluate(x_train, y_train, verbose=0)
    return Loss

In [42]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 16)                192       
                                                                 
 dense_5 (Dense)             (None, 1)                 17        
                                                                 
Total params: 209
Trainable params: 209
Non-trainable params: 0
_________________________________________________________________


In [43]:
def ant_colony_optimization(population_size=10, iterations=20, alpha=1, beta=2, evaporation_rate=0.5, Q=100):
    
    # initialize the pheromone matrix
    pheromone = np.ones((12, 16))
    
    
    # initialize the best weights and the loss
    best_weights = None
    best_loss = 99
    
    for i in range(iterations):
        # initialize the ant population and ant accuracies
        ants = []
        ant_loss = []
        
        for j in range(population_size):
            weights = np.zeros((12,16))
            for k in range(12):
                for l in range(16):
                    if np.random.rand() < pheromone[k][l]:
                        weights[k][l] = np.random.randn()
                        
            losses = fitness(weights)
            ants.append(weights)
            ant_loss.append(losses)
            
            
            if losses < best_loss:
                best_weights = weights
                best_loss = losses
                
                
        # update the pheromone matrix
        for k in range(11):
            for l in range(16):
                pheromone[k][l] = (1 - evaporation_rate) * pheromone[k][l]
                for j in range(population_size):
                    if ants[j][k][l] != 0:
                        pheromone[k][l] += Q / ant_loss[j] * ants[j][k][l]
       
        if i % 10 == 0:
            print("Iteration:", i, "Best Loss:", best_loss)
    
    return best_weights

In [44]:
best_weights = ant_colony_optimization()

Iteration: 0 Best Loss: 1.3502413034439087
Iteration: 10 Best Loss: 0.5170713663101196


In [47]:
best_weights = best_weights.flatten()
w0 = best_weights[:176].reshape((11, 16))
b0= np.array([0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.])

model.layers[0].set_weights([w0, b0])
model.compile(optimizer=optimizer, loss=loss)

In [49]:
import numpy as np
y_pred = model.predict(x_test)
y_pred = np.where(y_pred>=0.5,1,0)
from sklearn.metrics import accuracy_score
accuracy_score(y_pred,y_test)

47/47 [==============================] - 0s 1ms/step


0.7566666666666667